# 第2回講義 宿題

## 課題. MNISTをk-NNで学習せよ

### 注意
- homework関数を完成させて提出してください
    - 訓練データはtrain_X, train_y, テストデータはtest_Xで与えられます
    - train_Xとtrain_yをtrain_X, train_yとvalid_X, valid_yに分けるなどしてモデルを学習させてください
    - test_Xに対して予想ラベルpred_yを作り, homework関数の戻り値としてください\
- pred_yのtest_yに対する精度(F値)で評価します
- 全体の実行時間がiLect上で60分を超えないようにしてください
- homework関数の外には何も書かないでください

- k-NNの実装にscikit-learnなどのライブラリを使わないでください

次のコードが**事前**に実行されます

``` python
from __future__ import division
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split

import numpy as np

mnist = fetch_mldata('MNIST original')
mnist_X, mnist_y = shuffle(mnist.data, mnist.target.astype('int32'))

mnist_X = mnist_X / 255.0

train_X, test_X, train_y, test_y = train_test_split(mnist_X, mnist_y, test_size=0.2, random_state=??) # random_stateはこちらで与えます
```

次のセルのhomework関数を完成させて提出してください
- パッケージのインポートなども含めて、必要な物はすべて書いてください

In [ ]:
def homework(train_X, train_y, test_X):
    # WRITE ME
    from sklearn.base import BaseEstimator
    from sklearn.metrics.pairwise import cosine_similarity
    import bottleneck
    
    class knn_estimator(BaseEstimator):
        """k-nearest-neighbor classifier."""
        def __init__(self, k=1, sub_size=1000):
            self.k = k
            self.sub_size = sub_size

        def fit(self, X, y):
            self.X = X
            self.y = y
            return self

        def predict(self, X):
            # for the memory constriction, we split the testing set into small subsets
            sub_counts = int(np.ceil(X.shape[0] / self.sub_size))
            Xs = np.array_split(X, sub_counts)
            pred_y = []
            k = self.k
            for i in xrange(sub_counts):
                tmp_pred_y = np.zeros(Xs[i].shape[0])
                similarities = cosine_similarity(Xs[i], self.X)
                for j in xrange(Xs[i].shape[0]):
                    # candidates are 10 digits, i.e., 0,1,2,...,8,9
                    candidates = np.zeros(10)
                    # bottleneck partially sort the similarities, so we get k indexes with largest similarities in training set
                    knns = bottleneck.argpartsort(-similarities[j],n = k)[:k]
                    # voting for pred_y
                    for nn in knns:
                        candidates[train_y[nn]] += 1
                    # get pred_y by majority voting
                    tmp_pred_y[j] = np.argmax(candidates)
                pred_y.append(tmp_pred_y)
            pred_y = np.hstack(pred_y)
            return pred_y
    
    knn = knn_estimator(k=5, sub_size=1000)
    return pred_y

In [ ]:
from __future__ import division
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split

import numpy as np

mnist = fetch_mldata('MNIST original')
mnist_X, mnist_y = shuffle(mnist.data, mnist.target.astype('int32'))

mnist_X = mnist_X / 255.0

train_X, test_X, train_y, test_y = train_test_split(mnist_X, mnist_y, test_size=0.2)

def check_homework():
    pred_y = homework(train_X, train_y, test_X)
    return f1_score(test_y, pred_y, average='macro')

if 'homework' in globals():
    result = check_homework()
    print(result)
    if result > 0.95:
        print "No Error Occured! Your F1 score is 0.95~1.00"
    elif result > 0.90:
        print "No Error Occured! Your F1 score is 0.90~0.95"
    else:
        print "No Error Occured! Your F1 score is under 0.90"

In [1]:
from __future__ import division
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split

import numpy as np

mnist = fetch_mldata('MNIST original')
mnist_X, mnist_y = shuffle(mnist.data, mnist.target.astype('int32'))

mnist_X = mnist_X / 255.0

train_X, test_X, train_y, test_y = train_test_split(mnist_X, mnist_y, test_size=0.2)


from sklearn.base import BaseEstimator
from sklearn.metrics.pairwise import cosine_similarity
import bottleneck
class knn_estimator(BaseEstimator):
    """k-nearest-neighbor classifier."""
    def __init__(self, k=1, sub_size=1000):
        self.k = k
        self.sub_size = sub_size
        
    def fit(self, X, y):
        self.X = X
        self.y = y
        return self
    
    def predict(self, X):
        sub_counts = int(np.ceil(X.shape[0] / self.sub_size))
        Xs = np.array_split(X, sub_counts)
        pred_y = []
        k = self.k
        for i in xrange(sub_counts):
            tmp_pred_y = np.zeros(Xs[i].shape[0])
            similarities = cosine_similarity(Xs[i], self.X)
            for j in xrange(Xs[i].shape[0]):
                # candidates are 10 digits, i.e., 0,1,2,...,8,9
                candidates = np.zeros(10)
                # bottleneck partially sort the similarities, so we get k indexes with largest similarities in training set
                knns = bottleneck.argpartsort(-similarities[j],n = k)[:k]
                # voting for pred_y
                for nn in knns:
                    candidates[train_y[nn]] += 1
                # get pred_y by majority voting
                tmp_pred_y[j] = np.argmax(candidates)
            pred_y.append(tmp_pred_y)
        pred_y = np.hstack(pred_y)
        return pred_y
            

In [6]:
knn1 = knn_estimator(5, 1000)
knn1.fit(train_X, train_y)
pred_y = knn1.predict(test_X)

In [2]:
from sklearn.utils.estimator_checks import check_estimator
check_estimator(knn_estimator)

AssertionError: TypeError not raised by fit

In [ ]:
param_grid = [
  {'k': [1,3,5,10]},
 ]
from sklearn.grid_search import GridSearchCV
clf = GridSearchCV(knn_estimator(), param_grid, cv=5,
                       scoring='f1_macro')
clf.fit(train_X, train_y)

In [5]:
print(clf.best_params_)
for params, mean_score, scores in clf.grid_scores_:
    print("%0.3f (+/-%0.03f) for %r"
        % (mean_score, scores.std() * 2, params))
knn

{'k': 10}
0.532 (+/-0.617) for {'k': 1}
0.588 (+/-0.654) for {'k': 3}
0.657 (+/-0.671) for {'k': 5}
0.716 (+/-0.677) for {'k': 10}


In [ ]:
from __future__ import division
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split

import numpy as np

mnist = fetch_mldata('MNIST original')
mnist_X, mnist_y = shuffle(mnist.data, mnist.target.astype('int32'))

mnist_X = mnist_X / 255.0

train_X, test_X, train_y, test_y = train_test_split(mnist_X, mnist_y, test_size=0.2)

def check_homework():
    pred_y = homework(train_X, train_y, test_X)
    return f1_score(test_y, pred_y, average='macro')

if 'homework' in globals():
    result = check_homework()
    
    if result > 0.95:
        print "No Error Occured! Your F1 score is 0.95~1.00"
    elif result > 0.90:
        print "No Error Occured! Your F1 score is 0.90~0.95"
    else:
        print "No Error Occured! Your F1 score is under 0.90"